In [31]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import urllib2
from sklearn.linear_model import LogisticRegression

%matplotlib inline
import matplotlib.pyplot as plt

import statsmodels.api as sm
import pylab as pl
import numpy as np
from sklearn import cross_validation
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors, datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

In [32]:
urlForHigh = "http://www.indeed.com/jobs?q=data+scientist+%24130%2C000&l="
urlForLow  = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l="
page_start_from = '&start='    


df1 = pd.DataFrame()    
df2 = pd.DataFrame()   

for city in ('New+York%2C+NY', 'Chicago%2C+IL', 'San+Francisco%2C+CA', 'Austin%2C+TX',
            "Atlanta%2C+GA", "Los+Angeles%2C+CA", "Washington%2C+DC","Denver","Seattle","Austin","Dallas","San+Jose","Boston","Charlotte" ):
    for page in range(1,21): 
        page = (page-1) * 10  
        urlL = "%s%s%s%d" % (urlForLow, city, page_start_from, page) 
        r = requests.get(urlL)
        soupL = BeautifulSoup(r.content) 

        results = soupL.find_all("div", {'class' : ' row result'}) 
    
        for i in range(len(results)):
            title    = results[i].find ("a", {"class": "turnstileLink"}).get_text().strip()
            company  = results[i].find ("span", {"class": "company"}).get_text().strip()
            location = results[i].find ("span", {"class": "location"}).get_text().strip()
            summary  = results[i].find ("span", {"class": "summary"}).get_text().strip()        
        
        
            df1 = df1.append({"Company Name": company, "Job Title": title, "Location": location,
                            "Job Summary": summary}, ignore_index=True)

    

for city in ('New+York%2C+NY', 'Chicago%2C+IL', 'San+Francisco%2C+CA', 'Austin%2C+TX',
            "Atlanta%2C+GA", "Los+Angeles%2C+CA", "Washington%2C+DC","Denver","Seattle","Austin","Dallas","San+Jose","Boston","Charlotte" ):
    for page in range(1,21): 
        page = (page-1) * 10  
        urlH = "%s%s%s%d" % (urlForHigh, city, page_start_from, page) 
        r = requests.get(urlH)
        soupH = BeautifulSoup(r.content) 

        results = soupH.find_all("div", {'class' : ' row result'}) 
    
        for i in range(len(results)):
            title    = results[i].find ("a", {"class": "turnstileLink"}).get_text().strip()
            company  = results[i].find ("span", {"class": "company"}).get_text().strip()
            location = results[i].find ("span", {"class": "location"}).get_text().strip()
            summary  = results[i].find ("span", {"class": "summary"}).get_text().strip()        
           
        
            df2 = df2.append({"Company Name": company, "Job Title": title, "Location": location,
                            "Job Summary": summary}, ignore_index=True)


In [33]:
print df1.shape
print df2.shape

(2480, 4)
(2078, 4)


In [34]:
df1["Salary Class"]="Low" 
df1["Class"]=0
df2["Salary Class"]='High'
df2["Class"]=1

In [35]:
merge=[df1,df2]

In [36]:
merged = pd.concat(merge) # nice!!

In [37]:
df3=merged.drop_duplicates(["Company Name","Job Title","Location","Job Summary"],take_last=True)

/Users/bilindhajer/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead
  if __name__ == '__main__':


In [38]:
df3.to_csv("data.csv", encoding='utf-8')

In [39]:
data=pd.read_csv('data.csv')

In [40]:
del data['Unnamed: 0']

In [41]:
data

,Company Name,Job Summary,Job Title,Location,Salary Class,Class
0,Globant,Work alongside Sr Data Scientists in research....,"Ssr Data Scientist (R, Tableau, Python)","New York, NY",Low,0
1,Twitter,Strong algorithms and data structures backgrou...,Data Scientist (Deep Learning),"New York, NY",Low,0
2,New York Life Insurance Co,We are looking for a Data Scientist / Scientis...,Statistician/Data Scientist,"New York, NY",Low,0
3,Bloomberg,Bloomberg's Global Data division houses an inc...,Machine Learning Engineer,"New York, NY",Low,0
4,DataKind,A mid-career data scientist - You have a succe...,Data Scientist,"New York, NY",Low,0
5,Paperless Post,Build production systems to collect and proces...,Data Scientist,"New York, NY",Low,0
6,Memorial Sloan Kettering Cancer Center,"Working together with members in the group, th...",Bioinformatics Data Scientist - Cancer Genomic...,"New York, NY 10065 (Upper East Side area)",Low,0
7,DISH Network,A successful Data Scientist will have the foll...,Data Scientist,"New York, NY",Low,0
8,ShopKeep,"We have a mature data infrastructure, and we’d...",Data Scientist,"New York, NY 10013 (Tribeca area)",Low,0
9,Tapad,Tapad is looking for an experienced Data Scien...,Data Scientist,"New York, NY",Low,0


In [42]:
def binJob1(job):
    jobTitle = job.lower()
    if "data scientist" in jobTitle:
        return "Data scientist"
    elif "engineer" in jobTitle:
        return "Engineer"
    elif "marketing" in jobTitle:
        return "Marketing"
    elif "machine learning" in jobTitle:
        return "Machine learning"
    elif "software" in jobTitle:
        return "Software"
    elif "research" in jobTitle:
        return "Research"
    elif "consultant"in jobTitle:
        return "Consultant"
    elif "cyber" in jobTitle:
        return "Cyber"
    else:
        return "Other"
    
data["JobFunction"] = data["Job Title"].apply(binJob1)

In [43]:
def binJob2(job):
    jobTitle = job.lower()
    
    
    if ("sr" in jobTitle) | ("senior" in jobTitle):
        return 'Senior'
    elif "manager" in jobTitle:
        return "Manager"
    elif "principle" in jobTitle:
        return "Principle"
    elif "chief" in jobTitle:
        return "Chief"
    elif "lead" in jobTitle:
        return "Lead"
    elif "director"in jobTitle:
        return "Director"
    elif "entry" in jobTitle:
        return "Entry"
    elif "fellow" in jobTitle:
        return "Fellow"
    elif "assistant" in jobTitle:
        return "Assistant"
    else:
        return "Other"
    
data["JobSeniority"] = data["Job Title"].apply(binJob2)

In [44]:

def locationToRegion(location):
    if ("NY" in location)|("MA" in location)|("NJ" in location):
        return "Northeast"
    elif ("DC" in location)|("VA" in location)|("MD" in location):
        return "DMV"
    elif "CA"  in location:
        return "California"
    elif ("IL" in location)|("CO" in location):
        return "Central"
    elif "WA" in location:
        return "Northwest"
    elif "TX" in location:
        return "Texas"
    elif ("NC" in location)|("GA" in location):
        return "Southeast"
    else:
        return "Other"
_


data['region'] = data['Location'].apply(locationToRegion)
        

In [45]:
del data["Data Scientist"]

KeyError: 'Data Scientist'

In [46]:
dummy_jobFunction = pd.get_dummies(data['JobFunction'], prefix='JobFunction')
dummy_Seniority = pd.get_dummies(data['JobSeniority'], prefix='JobSeniority')
dummy_Region = pd.get_dummies(data['region'], prefix='Region')

In [47]:
data

,Company Name,Job Summary,Job Title,Location,Salary Class,Class,JobFunction,JobSeniority,region
0,Globant,Work alongside Sr Data Scientists in research....,"Ssr Data Scientist (R, Tableau, Python)","New York, NY",Low,0,Data scientist,Senior,Northeast
1,Twitter,Strong algorithms and data structures backgrou...,Data Scientist (Deep Learning),"New York, NY",Low,0,Data scientist,Other,Northeast
2,New York Life Insurance Co,We are looking for a Data Scientist / Scientis...,Statistician/Data Scientist,"New York, NY",Low,0,Data scientist,Other,Northeast
3,Bloomberg,Bloomberg's Global Data division houses an inc...,Machine Learning Engineer,"New York, NY",Low,0,Engineer,Other,Northeast
4,DataKind,A mid-career data scientist - You have a succe...,Data Scientist,"New York, NY",Low,0,Data scientist,Other,Northeast
5,Paperless Post,Build production systems to collect and proces...,Data Scientist,"New York, NY",Low,0,Data scientist,Other,Northeast
6,Memorial Sloan Kettering Cancer Center,"Working together with members in the group, th...",Bioinformatics Data Scientist - Cancer Genomic...,"New York, NY 10065 (Upper East Side area)",Low,0,Data scientist,Other,Northeast
7,DISH Network,A successful Data Scientist will have the foll...,Data Scientist,"New York, NY",Low,0,Data scientist,Other,Northeast
8,ShopKeep,"We have a mature data infrastructure, and we’d...",Data Scientist,"New York, NY 10013 (Tribeca area)",Low,0,Data scientist,Other,Northeast
9,Tapad,Tapad is looking for an experienced Data Scien...,Data Scientist,"New York, NY",Low,0,Data scientist,Other,Northeast


In [48]:
cols_to_keep = ['Company Name', 'Job Summary', 'Job Title',"Location","Salary Class","Class"]
merge = data[cols_to_keep].join(dummy_jobFunction.ix[:,])
merge1=merge.join(dummy_Seniority.ix[:, ])
merged=merge1.join(dummy_Region.ix[:, ])

merged

# good job on the merging and binning! 

,Company Name,Job Summary,Job Title,Location,Salary Class,Class,JobFunction_Consultant,JobFunction_Cyber,JobFunction_Data scientist,JobFunction_Engineer,...,JobSeniority_Principle,JobSeniority_Senior,Region_California,Region_Central,Region_DMV,Region_Northeast,Region_Northwest,Region_Other,Region_Southeast,Region_Texas
0,Globant,Work alongside Sr Data Scientists in research....,"Ssr Data Scientist (R, Tableau, Python)","New York, NY",Low,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
1,Twitter,Strong algorithms and data structures backgrou...,Data Scientist (Deep Learning),"New York, NY",Low,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,New York Life Insurance Co,We are looking for a Data Scientist / Scientis...,Statistician/Data Scientist,"New York, NY",Low,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,Bloomberg,Bloomberg's Global Data division houses an inc...,Machine Learning Engineer,"New York, NY",Low,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,DataKind,A mid-career data scientist - You have a succe...,Data Scientist,"New York, NY",Low,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
5,Paperless Post,Build production systems to collect and proces...,Data Scientist,"New York, NY",Low,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
6,Memorial Sloan Kettering Cancer Center,"Working together with members in the group, th...",Bioinformatics Data Scientist - Cancer Genomic...,"New York, NY 10065 (Upper East Side area)",Low,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
7,DISH Network,A successful Data Scientist will have the foll...,Data Scientist,"New York, NY",Low,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
8,ShopKeep,"We have a mature data infrastructure, and we’d...",Data Scientist,"New York, NY 10013 (Tribeca area)",Low,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
9,Tapad,Tapad is looking for an experienced Data Scien...,Data Scientist,"New York, NY",Low,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [49]:
#Base case is set to be that a Senior Consultant at California. The intercept of this case is 1, which
#represents a high income. Since it't the base case, we need to drop these variables.

del merged["JobSeniority_Senior"]
del merged["Region_California"]
del merged["JobFunction_Consultant"]

In [50]:
merged['intercept'] = 1

In [51]:

train_cols = merged.columns[6:]   
logit = sm.Logit(merged['Class'], merged[train_cols])
result = logit.fit_regularized() # interpret!



Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.509106663676
            Iterations: 189
            Function evaluations: 189
            Gradient evaluations: 189


In [52]:
print result.summary() # interpret!

                           Logit Regression Results                           
Dep. Variable:                  Class   No. Observations:                 2884
Model:                          Logit   Df Residuals:                     2859
Method:                           MLE   Df Model:                           24
Date:                Mon, 04 Jul 2016   Pseudo R-squ.:                  0.2109
Time:                        17:22:41   Log-Likelihood:                -1468.3
converged:                       True   LL-Null:                       -1860.7
                                        LLR p-value:                3.185e-150
                                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------
JobFunction_Cyber                1.3689      1.284      1.066      0.287        -1.149     3.886
JobFunction_Data scientist       0.2782      0.509      0.546      0.585     

In [53]:
print np.exp(result.params) # interpret!

JobFunction_Cyber               3.931071e+00
JobFunction_Data scientist      1.320758e+00
JobFunction_Engineer            4.119547e+00
JobFunction_Machine learning    5.486084e+00
JobFunction_Marketing           5.707494e-01
JobFunction_Other               1.478058e+00
JobFunction_Research            4.955160e-01
JobFunction_Software            1.285111e+01
JobSeniority_Assistant          1.136674e-01
JobSeniority_Chief              1.610985e+01
JobSeniority_Director           4.185726e+00
JobSeniority_Entry              4.568217e-13
JobSeniority_Fellow             1.710461e-13
JobSeniority_Lead               1.893388e+00
JobSeniority_Manager            1.358062e+00
JobSeniority_Other              3.536369e-01
JobSeniority_Principle          1.166766e+10
Region_Central                  2.036997e-01
Region_DMV                      5.118065e-01
Region_Northeast                1.169798e+00
Region_Northwest                7.063232e-01
Region_Other                    2.058909e-06
Region_Sou

In [54]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

# please interpret!

                                  2.5%       97.5%            OR
JobFunction_Cyber             0.317100   48.733193  3.931071e+00
JobFunction_Data scientist    0.486930    3.582446  1.320758e+00
JobFunction_Engineer          1.498453   11.325458  4.119547e+00
JobFunction_Machine learning  1.755734   17.142184  5.486084e+00
JobFunction_Marketing         0.135456    2.404869  5.707494e-01
JobFunction_Other             0.546694    3.996121  1.478058e+00
JobFunction_Research          0.172580    1.422738  4.955160e-01
JobFunction_Software          3.336818   49.493544  1.285111e+01
JobSeniority_Assistant        0.014247    0.906875  1.136674e-01
JobSeniority_Chief            2.019954  128.481861  1.610985e+01
JobSeniority_Director         2.454794    7.137180  4.185726e+00
JobSeniority_Entry            0.000000         inf  4.568217e-13
JobSeniority_Fellow           0.000000         inf  1.710461e-13
JobSeniority_Lead             1.141394    3.140826  1.893388e+00
JobSeniority_Manager     

In [55]:
train_cols = merged.columns[6:]

logit = sm.Logit(merged['Class'], merged[train_cols])
result = logit.fit_regularized()

result.summary()

# please intepret! what do these coefficients mean?

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.509106663676
            Iterations: 189
            Function evaluations: 189
            Gradient evaluations: 189


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  Class   No. Observations:                 2884
Model:                          Logit   Df Residuals:                     2859
Method:                           MLE   Df Model:                           24
Date:                Mon, 04 Jul 2016   Pseudo R-squ.:                  0.2109
Time:                        17:22:49   Log-Likelihood:                -1468.3
converged:                       True   LL-Null:                       -1860.7
                                        LLR p-value:                3.185e-150
================================================================================================
                                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------
JobFunction_Cyber                1.3689      1.284      1.066      0.287        -1.149     3.886
JobFunction_Data scientist       0.2782      0.509      0.546      0.585        -0.720     1.276
JobFunction_Engineer             1.4157      0.516      2.744      0.006         0.404     2.427
JobFunction_Machine learning     1.7022      0.581      2.928      0.003         0.563     2.842
JobFunction_Marketing           -0.5608      0.734     -0.764      0.445        -1.999     0.877
JobFunction_Other                0.3907      0.507      0.770      0.441        -0.604     1.385
JobFunction_Research            -0.7022      0.538     -1.305      0.192        -1.757     0.353
JobFunction_Software             2.5534      0.688      3.711      0.000         1.205     3.902
JobSeniority_Assistant          -2.1745      1.060     -2.052      0.040        -4.251    -0.098
JobSeniority_Chief               2.7794      1.059      2.624      0.009         0.703     4.856
JobSeniority_Director            1.4317      0.272      5.258      0.000         0.898     1.965
JobSeniority_Entry             -28.4145   6.58e+05  -4.32e-05      1.000     -1.29e+06  1.29e+06
JobSeniority_Fellow            -29.3968   8.75e+05  -3.36e-05      1.000     -1.71e+06  1.71e+06
JobSeniority_Lead                0.6384      0.258      2.472      0.013         0.132     1.144
JobSeniority_Manager             0.3061      0.216      1.417      0.156        -0.117     0.729
JobSeniority_Other              -1.0395      0.109     -9.558      0.000        -1.253    -0.826
JobSeniority_Principle          23.1801   7.51e+04      0.000      1.000     -1.47e+05  1.47e+05
Region_Central                  -1.5911      0.168     -9.448      0.000        -1.921    -1.261
Region_DMV                      -0.6698      0.173     -3.862      0.000        -1.010    -0.330
Region_Northeast                 0.1568      0.120      1.307      0.191        -0.078     0.392
Region_Northwest                -0.3477      0.158     -2.203      0.028        -0.657    -0.038
Region_Other                   -13.0933   1016.781     -0.013      0.990     -2005.947  1979.760
Region_Southeast                -1.5957      0.215     -7.429      0.000        -2.017    -1.175
Region_Texas                    -2.1385      0.197    -10.847      0.000        -2.525    -1.752
intercept                        0.0058      0.511      0.011      0.991        -0.996     1.008
================================================================================================
"""

In [56]:
from sklearn import linear_model, datasets
logreg = linear_model.LogisticRegression(C=1e5)
result2=logreg.fit(X, y)

# X is not defined here, therefore this cell, nor any of those below will run!

result2




NameError: name 'X' is not defined

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

NameError: name 'X' is not defined

In [58]:
X_test["actualHigh"] = y_test
X_test.head()
dfTrain = X_test

print dfTrain.columns

dfTrain['predictedHigh'] = result.predict( dfTrain[ dfTrain.columns[0:24] ] )

NameError: name 'y_test' is not defined

In [59]:
def scalePredictor(prob, threshold):
    if( prob >= threshold ):
        return 1
    return 0

In [60]:
i = 0.20
while i <= 0.50:
    
    dfTrain[ 'predictedHigh_{}'.format(i) ] = dfTrain.predictedHigh.apply(
        lambda prob: scalePredictor(prob,i) )
    
    print pd.crosstab(
        dfTrain['actualHigh'],
        dfTrain['predictedHigh_{}'.format(i)], 
        rownames=['High']
    )
    i += 0.01

NameError: name 'dfTrain' is not defined

In [61]:
import numpy as np
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm
from sklearn import metrics

scores =cross_validation.cross_val_score(model, X, y, cv=6)
print "Cross-validated Scores:", scores
# Make cross validated predictions
predictions = cross_validation.cross_val_predict(model, X, y, cv=6)
#plt.scatter(y, predictions)
accuracy = metrics.accuracy_score(y, predictions)
print "Cross-Predicted Accuracy:", accuracy

NameError: name 'model' is not defined